# Download Images

In [5]:
import sys, os, io, datetime
import json
import random
import requests
import zipfile
import glob
import re
import pandas as pd
from slugify import slugify
from tqdm.notebook import tqdm
from html import escape
from collections import defaultdict

In [6]:
import cdli
import languages

In [7]:
pd.set_option("display.max_columns", None)

In [8]:
supported_langs = set(["akk", "sux"])

In [35]:
photos_dir = os.path.abspath("/Volumes/FrankDisk/cdli_photos")
os.makedirs(photos_dir, exist_ok=True)
photos_dir

'/Volumes/FrankDisk/cdli_photos'

In [37]:
bad_photos_dir = os.path.abspath("/Volumes/FrankDisk/cdli_bad_photos")
os.makedirs(bad_photos_dir, exist_ok=True)
bad_photos_dir

'/Volumes/FrankDisk/cdli_bad_photos'

## Get the Human Transliterations

In [38]:
all_publications = cdli.get_atf()

Parsing atf


## Start a web session

In [39]:
http = requests.Session()

In [40]:
http.get("https://cdli.ucla.edu").text[:100]

'<!DOCTYPE html>\n<!--[if IEMobile 7]><html class="iem7"  lang="en" dir="ltr"><![endif]-->\n<!--[if lte'

## Download the photos

In [41]:
def pp(pid): return f"{photos_dir}/{pid}.jpg"
def bad_pp(pid): return f"{bad_photos_dir}/{pid}.jpg"

In [42]:
def download_photo(pid):
    resp = http.get(f"https://cdli.ucla.edu/dl/photo/{pid}.jpg")
    if resp.status_code != 200:
        with open(bad_pp(pid), "wb") as f:
            f.write(resp.content)
    else:
        with open(pp(pid), "wb") as f:
            f.write(resp.content)

download_photo("P238563")

In [43]:
needed_photos = { x.id for x in all_publications if len(str(x.id)) > 1 and not (os.path.exists(pp(x.id)) or os.path.exists(bad_pp(x.id))) }
len(needed_photos), "needed"

(120489, 'needed')

In [44]:
for p in tqdm(needed_photos):
    download_photo(p)

  0%|          | 0/120489 [00:00<?, ?it/s]

In [21]:
def header(title, f):
    f.write(f"<html>\n<head>\n<title>{escape(title)}</title>\n<meta name='viewport' content='width=device-width, initial-scale=1'>\n<style>{style}</style></head>\n")
    f.write(f"<body>\n<div class='content'><h1>{escape(title)}</h1>\n")
    f.write(f"<p class='otitle'>ML Translations by <a href=\"https://huggingface.co/praeclarum/cuneiform\">praeclarum/cuneiform</a></p>\n")
    
def footer(f):
    f.write(f"</div></body>\n</html>")

In [22]:
def start_page(path, title):
    file_path = get_page_file_path(path)
    file_dir = os.path.dirname(file_path)
    os.makedirs(file_dir, exist_ok=True)
    f = open(file_path, "wt")
#     print(f"Writing {path} at {file_path}")
    header(title, f)
    return f

def end_page(f):
    footer(f)

In [23]:
def output_browser(path, pubs, ignore_dims, f):
    next_pages = []
    if len(pubs) == 0:
        return next_pages
    f.write(f"<section>\n")
    for dname, dselect in browser_dimensions:
        if dname in ignore_dims:
            continue
        vgroups = defaultdict(lambda: [])
        for p in pubs:
            for v in dselect(p):
                vgroups[v].append(p)
        if len(vgroups) < 2:
            continue
        f.write(f"<h1>{escape(dname)}</h1>\n")    
        for gv in vgroups.keys():
            gpubs = vgroups[gv]
            if len(gpubs) > 0:
                next_pages.append((dname, gv, gpubs))
                f.write(f"<a href='{path}/{gv}/'>{len(gpubs)} {escape(gv)}</a>\n")
    f.write(f"</section>\n")
    f.write(f"<section>\n")
    max_on_page = 200
    if len(pubs) <= max_on_page or len(next_pages) == 0:
        for p in pubs:
            output_pub(p, f)        
    else:
        f.write(f"<p>{len(pubs)} publications. Narrow the list to less than {max_on_page} by choosing links above.</p>\n")
    f.write(f"</section>\n")
    return next_pages

def output_browser_page(path, dim_value, pubs, ignore_dims):
    with start_page(path + "/index", f"{len(pubs)} {escape(dim_value)}") as f:
        next_pages = output_browser(path, pubs, ignore_dims, f)
        end_page(f)
    for gk, gv, gpubs in next_pages:
        ignores = set(ignore_dims)
        ignores.add(gk)
        output_browser_page(f"{path}/{gv}", gv, gpubs, ignores)


In [24]:
style = """
body { font-family: sans-serif; }
div.content { margin: 0 auto; max-width: 512px; }
h1 { text-align: center; }
p.src { font-size: 80%; font-style: italic;}
.otitle { text-align: center; }
h1.otitle { margin-top: 1em; padding-top: 1em; border-top:solid 2px rgba(128,128,128,0.5); }
.langid {
    font-size: 80%; font-weight: bold; 
    background-color: rgba(128,128,128,0.25); border-radius:0.2em;
    padding: 0 0; min-width: 5em; display: inline-block;
    text-align: center; margin-right: 1em;
}
.lang-akkts, .lang-elxts, .lang-suxts, .lang-akk, .lang-elx, .lang-sux {
    font-style: italic; font-size: 80%;
    opacity: 0.75;
    }
.lang-en { opacity: 0.5; font-size: 80%; }
.lang-ml_en {}
.line:hover { background-color: rgba(255, 255, 128, 1.0); }
.text { margin-top: 2em; line-height: 1.2em; }
"""

In [25]:
languages.old_languages

{'akk': 'Akkadian',
 'sux': 'Sumerian',
 'qpn': 'Proper Nouns',
 'arc': 'Aramaic',
 'elx': 'Elamite',
 'grc': 'Greek',
 'peo': 'Old Persian',
 'ug': 'Ugaritic'}

In [26]:
languages.cuneiform_text_to_unicode("ki na-lu5-ta ur-nigar{gar} szu ba-ti iti masz-da3-gu7 mu us2-sa ki-masz{ki} ba-hul", "sux")

'𒆠 𒈾lu5𒋫 𒌨nigar{𒃻} szu 𒁀𒋾 iti maszda3gu7 𒈬 us2𒊓 𒆠masz{𒆠} 𒁀hul'

In [28]:
languages.cuneiform_text_to_unicode("iti", "sux")

'iti'

In [35]:
language_sort = {
    "akk": 0,
#     "akkts": 1,
#     "elx": 2,
#     "elxts": 3,
    "sux": 4,
#     "suxts": 5,
    "ml_en": 100,
    "en": 1000,
#     "fr": 1001,
}

bad_translators = {"uncertain", "NaN", "no translation", "", "check"}

li_start_re = re.compile(r"^\[?_?\d+(/\d+)?\((asz|ban2|barig|disz|gesz2|iku|u)(@c)?\)\]?")

def looks_like_li(line, lang):
    return li_start_re.match(line)

def lines_to_paragraphs(lines, lang):
    paragraphs = []
    for iline, line in enumerate(lines):
        if looks_like_li(line.text, lang):
            paragraphs.append(("li", iline, iline+1))
        else:
            if len(paragraphs) > 0 and paragraphs[-1][0] == "p":
                p = paragraphs[-1]
                p = (p[0], p[1], p[2]+1)
                paragraphs[-1] = p
            else:
                paragraphs.append(("p", iline, iline+1))
    return paragraphs

def paragraphs_to_html(paragraphs, text_lines):
    html = []
    for tag, line_start_index, line_end_index in paragraphs:
        html.append(f"<{tag}>")
        for i in range(line_start_index, line_end_index):
            html.append(f"<span class='line line-{i}'>{escape(text_lines[i])}</span>\n")
        html.append(f"</{tag}>\n")
    return "".join(html)

def output_pub(p, f):
    f.write(f"<h1 class='otitle'>P{p.id:06}: {' and '.join(cdli.get_genres(p.genre))} {cdli.get_object_type(p.object_type)}</h1>\n")
    f.write(f"<p class='otitle'>{p.period} <a href='https://cdli.ucla.edu/search/archival_view.php?ObjectID=P{p.id:06}'>CDLI</a></p>\n")
    areas_with_lines = [x for x in p.text_areas if any(x.lines)]
    for a in areas_with_lines:
        f.write(f"<section>\n")
        if len(areas_with_lines) > 1:
            f.write(f"<h1>{escape(a.name)}</h1>\n")
        show_lines = False
        if show_lines:
            f.write("<table>")
            for l in a.lines:
                f.write("<tr>")
                f.write(f"<td>{escape(l.text)}</td>\n")
                if "ml_en" in l.languages:
                    f.write(f"<td>{escape(l.languages['ml_en'])}</td>\n")
                else:
                    f.write(f"<td></td>\n")
                f.write("</tr>")
            f.write("</table>")
        else:
            paragraphs = lines_to_paragraphs(a.lines, p.language)
            texts = {p.language: paragraphs_to_html(paragraphs, [l.text for l in a.lines])}
            langs = set()
            for l in a.lines:
                for lang in l.languages:
                    if lang in language_sort:
                        langs.add(lang)
            for lang in langs:
                texts[lang] = paragraphs_to_html(paragraphs, [l.languages[lang] if lang in l.languages else " " for l in a.lines])
            langs.add(p.language)
#             f.write(f"<p><pre>{escape(repr(paragraphs))}</pre></p> ")
#             if "akkts" in langs and "akk" in langs:
#                 langs.remove("akk")
#             if "suxts" in langs and "sux" in langs:
#                 langs.remove("sux")
            langs = sorted(list(langs), key=lambda x:language_sort[x])
            for lang in langs:
                f.write(f"<div class='lang-{lang} text'><span class='langid'>{lang}</span> ")
                f.write(texts[lang])
                if lang == tgt_lang and p.translation_source not in bad_translators:
                    f.write(f"<span class='human-translation-credit'>(translated by {escape(p.translation_source)})</credit>")
                f.write(f"</div>\n")
        f.write(f"</section>\n")

In [36]:
#!rm -r /Users/fak/Dropbox/Projects/CuneiformTranslators/dist

In [37]:


with start_page("/404", "Not Found") as f:
    f.write(f"<p>The page you are looking for is not here.</p>\n")
    footer(f)
    
print("Writing /")
with start_page("/index", "ML Cuneiform Translations") as f:
    pubs = translated_pubs
    by_lang = defaultdict(lambda: [])
    for p in pubs:
        by_lang[p.language].append(p)
    f.write(f"<section><h1>language</h1></section>")
    next_pages = []
    for lang in sorted(list(by_lang.keys())):
        gpubs = by_lang[lang]
        f.write(f"<a href='/{lang}/'>{len(gpubs)} {escape(lang)}</a>\n")
        next_pages.append(("language", lang, gpubs))
    f.write(f"</section>")
    next_pages.extend(output_browser("", pubs, "", f))
    footer(f)
    
for gk, gv, gpubs in tqdm(next_pages):
    output_browser_page(f"/{gv}", gv, gpubs, set([gk]))
    
for f in glob.glob("../dist/*"):
    print(f)

Writing /


  0%|          | 0/52 [00:00<?, ?it/s]

../dist/akk
../dist/omen
../dist/other-genre
../dist/tablet
../dist/index.html
../dist/lexical
../dist/uncertain
../dist/hellenistic
../dist/ur-iii
../dist/barrel
../dist/private-votive
../dist/old-akkadian
../dist/prism
../dist/404.html
../dist/cone
../dist/ed-i-ii
../dist/letter
../dist/other-period
../dist/early-neo-babylonian
../dist/old-babylonian
../dist/ed-iiib
../dist/seal
../dist/other-object
../dist/neo-babylonian
../dist/astronomical
../dist/early-old-babylonian
../dist/bulla
../dist/neo-assyrian
../dist/old-assyrian
../dist/achaemenid
../dist/ebla
../dist/sux
../dist/administrative
../dist/middle-assyrian
../dist/brick
../dist/vessel
../dist/sealing
../dist/ritual
../dist/royal-monumental
../dist/mathematical
../dist/prayer-incantation
../dist/fake
../dist/middle-babylonian
../dist/lagash-ii
../dist/legal
../dist/middle-hittite
../dist/school
../dist/cylinder
../dist/tag
../dist/middle-elamite
../dist/block
../dist/literary
../dist/scientific
../dist/ed-iiia
